<a href="https://colab.research.google.com/github/studnt001/fall8w1-600/blob/main/mz_week5_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret

In [2]:
from pycaret.utils import version
version()

'3.3.2'

In [3]:
import pandas as pd

In [4]:
print(pd.__version__)

2.1.4


In [5]:
from google.colab import files

In [6]:
uploaded = files.upload()

Saving prepped_churn_data(in).csv to prepped_churn_data(in).csv


In [20]:
import io
df2 = pd.read_csv(io.BytesIO(uploaded['prepped_churn_data(in).csv']), index_col='customerID')

In [21]:
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [23]:
automl = setup(df2, target='Churn')

,Description,Value
0,Session id,6439
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7032, 8)"
4,Transformed data shape,"(7032, 8)"
5,Transformed train set shape,"(4922, 8)"
6,Transformed test set shape,"(2110, 8)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


In [24]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7944,0.8303,0.5137,0.6434,0.5698,0.4372,0.4428,0.8330
ridge,Ridge Classifier,0.7934,0.8159,0.4503,0.6660,0.5356,0.4094,0.4232,0.0380
ada,Ada Boost Classifier,0.7893,0.8298,0.4862,0.6416,0.5515,0.4172,0.4252,0.2480
gbc,Gradient Boosting Classifier,0.7883,0.8335,0.4709,0.6410,0.5415,0.4082,0.4172,0.8420
lda,Linear Discriminant Analysis,0.7863,0.8159,0.4885,0.6259,0.5480,0.4110,0.4168,0.0590
xgboost,Extreme Gradient Boosting,0.7783,0.8121,0.4816,0.6055,0.5357,0.3927,0.3977,0.1340
rf,Random Forest Classifier,0.7726,0.8033,0.4625,0.5929,0.5184,0.3730,0.3785,0.9250
lightgbm,Light Gradient Boosting Machine,0.7725,0.8195,0.4717,0.5922,0.5236,0.3769,0.3820,0.2980
knn,K Neighbors Classifier,0.7710,0.7378,0.4450,0.5932,0.5073,0.3624,0.3694,0.0740
svm,SVM - Linear Kernel,0.7696,0.7571,0.4548,0.5905,0.5058,0.3613,0.3704,0.0620


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [26]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=6439, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
df2.iloc[-2:-1].shape

(1, 8)

In [29]:
predict_model(best_model, df2.iloc[-2:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,1.0000,0,1.0000,1.0000,1.0000,nan,0.0000


,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,TC_t_ratio,Churn,prediction_label,prediction_score
customerID,,,,,,,,,,
8361-LTMKD,4,1,2,3,74.400002,306.600006,76.650002,1,1,0.5844


In [30]:
save_model(best_model, 'LR')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['tenure', 'PhoneService',
                                              'Contract', 'PaymentMethod',
                                              'MonthlyCharges', 'TotalCharges',
                                              'TC_t_ratio'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categori...
                                                         

In [31]:
import pickle

with open('LR_model.pk', 'wb') as f:
  pickle.dump(best_model, f)

In [32]:
with open('LR_model.pk', 'rb') as f:
  loaded_model = pickle.load(f)

In [33]:
new_data = df2.iloc[-2:-1].copy()
new_data.drop('Churn', axis=1, inplace=True)
loaded_model.predict(new_data)

array([1], dtype=int8)

In [34]:
loaded_lR = load_model('LR')

Transformation Pipeline and Model Successfully Loaded


In [35]:
predict_model(loaded_lR, new_data)

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,TC_t_ratio,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4,1,2,3,74.400002,306.600006,76.650002,1,0.5844


In [36]:
from IPython.display import Code

Code('predict_churn.py')

predict_churn.py

In [37]:
import pandas as pd
from pycaret.classification import predict_model, load_model

In [41]:
uploaded_n = files.upload()

Saving new_churn_data(in).csv to new_churn_data(in).csv


In [45]:
def load_data(filepath):
  df = pd.read_csv(filepath, index_col='customerID')
  return df

def make_predictions(df):
  model = load_model('LR')
  predictions = predict_model(model, data=df)
  print(predictions.columns)
  if 'prediction_label' in predictions.columns:
    predictions.rename(columns={'prediction_label': 'Churn_prediction'}, inplace=True)
    return predictions['Churn_prediction']
  else:
    raise KeyError("The 'prediction_label' column was not found in the predictions DataFrame")

if __name__ == "__main__":
  df = load_data('new_churn_data(in).csv')
  predictions = make_predictions(df)
  print('predictions:')
  print(predictions)

Transformation Pipeline and Model Successfully Loaded


Index(['tenure', 'PhoneService', 'Contract', 'PaymentMethod', 'MonthlyCharges',
       'TotalCharges', 'TC_t_ratio', 'prediction_label', 'prediction_score'],
      dtype='object')
predictions:
customerID
9305-CKSKC    0
1452-KNGVK    0
6723-OKKJM    0
7832-POPKP    0
6348-TACGU    0
Name: Churn_prediction, dtype: int64


Summary:
I chose to use Google Colab for this project instead of Jupyter Notebook because I had some dependency issues while trying to load the pycaret library into a virtual environment in Jupyter Lab. In this notebook created in Google Colab, first I installed and imported the packages needed including pandas and pycaret. Next I set up the autoML using pycaret's setup function and then run to find the best model. The best_model object uses accuracy by default, for this assignment, I will keep accuracy as the chosen metric. Once the model has been chosen, then I will save it to be used in a .py file. This is done by using pycaret's pickle function. Then I test that the pycaret model has been saved successfully and that the predictions work. Lastly, the model will be used in a .py file to read new data and make a prediction. The .py file is created by creating a way to load in new data and use the best model, which is Logistic Regression in this case, to make predictions on if customers are likely to churn or not given their customer and contract information. When the .py file is ran with the new data, the values are [0, 0, 0, 0, 0]. The true values are [1, 0, 0, 1, 0], which means there are two false negatives, where the LR model predicts that the customer will not churn, when in reality they will churn.